In [1]:
import pandas as pd, plotly.express as px, plotly.io as pio
pio.templates.default = 'plotly_white'

# Spend Analysis

Spend analysis is the practice of analyzing procurement spend to decrease costs, increase efficiency, or improve supplier relationships. In this notebook, we will cover different types of spend anaysis, including category spend, supplier spend, tail spend, item spend, and payment terms analysis.



### Example: Spend Data from an American Manufacturer

We will use spend data from an American manufacturer that was created with real data which has been anonymized.  The original dataset was distributed over multiple database tables but has been merged into a single view to facilitate analysis with Pandas and Plotly.

In [3]:
df = pd.read_csv("https://lscm.s3.amazonaws.com/machine_learning/05_spend_analysis.csv", parse_dates=["Date"])
df.head()

,InvoiceID,Date,ItemID,Quantity,UnitPrice,Discount,DueDays,InvoiceAmount,Category,Commodity,Site,VendorID,Name,Country,ZIP
0,466208,2012-06-28,205225,1,293.602548,0.0,65,293.60,Indirect,Maintenance & Repair,St. Louis,21,Greenelectronics Corporation,MX,Unknown
1,466208,2012-06-28,205225,1,3310.838662,0.0,65,3310.84,Indirect,Maintenance & Repair,St. Louis,21,Greenelectronics Corporation,MX,Unknown
2,466208,2012-06-28,205225,1,425.916545,0.0,65,425.92,Indirect,Maintenance & Repair,St. Louis,21,Greenelectronics Corporation,MX,Unknown
3,466208,2012-06-28,205225,1,263.509427,0.0,65,263.51,Indirect,Maintenance & Repair,St. Louis,21,Greenelectronics Corporation,MX,Unknown
4,466208,2012-06-28,205225,1,127.363880,0.0,65,127.36,Indirect,Maintenance & Repair,St. Louis,21,Greenelectronics Corporation,MX,Unknown


## Supplier Spend Analysis

Supplier spend analysis is the task of identifying the amount of spend coming from critical vendors. Different types of vendor reports visualize spend insights in several ways: by vendor, by category, and by geography. 


### Who are the top 20 vendors by value?

Every procurement organization must know its most important vendors by spend volume, so let us take a look at the top 20.

In [4]:
(px.bar(data_frame=(df[["Name","InvoiceAmount","Category"]]
                     .groupby(["Name","Category"])
                     .sum()
                     .reset_index()
                     .sort_values("InvoiceAmount")
                     .iloc[-20:]),
       y="Name", 
       x="InvoiceAmount",
       color="Category",
       orientation="h")
.update_layout(height=500)
)

## Which commodity are we sourcing from which company?

There are usually many multiple vendors across many business units, which offers space for consolidation. The aim is to reduce the number of vendors in each category but also ensure that important categories are not dominated by one single vendor.

In [6]:
(px.bar(
    data_frame=(df[["Commodity","Name","InvoiceAmount"]]
                .groupby(["Commodity","Name"])
                .sum()
                .reset_index()
                .sort_values("InvoiceAmount")),
    x="InvoiceAmount",
    y="Commodity",
    color="Name")
).update_layout(
    height=700
)

### Where are the 100 largest suppliers located in the U.S.?

As the largest production sites are located in the U.S., let us take look at where the 100 largest U.S. based suppliers are located. For this, we need to extract latitude and logitude info from the zip code.

In [5]:
!pip install pgeocode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pgeocode
nomin = pgeocode.Nominatim('us')
(px.scatter_mapbox(
    data_frame = (df.loc[lambda x: x.Country=="US",["Name","ZIP","InvoiceAmount"]]
            .groupby(["Name","ZIP"])
            .sum()
            .sort_values("InvoiceAmount").iloc[-100:]
            .reset_index()
            .assign(Latitude = lambda x : nomin.query_postal_code(x.ZIP.values).latitude)
            .assign(Longitude = lambda x : nomin.query_postal_code(x.ZIP.values).longitude)
            .dropna()
        ),
    lat="Latitude", 
    lon="Longitude",
    size = "InvoiceAmount",
    hover_data=["Name", "InvoiceAmount"],
    zoom=3, height=500)
).update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":0,"l":0,"b":0}
)

## Category Spend Analysis

Allocating spend consistently into categories makes the data easier to navigate, interpret, and understand. When purchase organizations have a high-level overview of spend by category, it is easier to identify categories that help in delivering savings and to realize which projects bring strategic importance to the organization. 

### Which categories do we spend most money on?

A procurement organization should always have an overview of the total spend by category and keep an eye on spend that is not assigned to a category. 

In [8]:
(px.bar(
    data_frame=(df[["Category","Commodity","InvoiceAmount"]]  
                .groupby(["Commodity","Category"])
                     .sum()
                     .reset_index()
                     .sort_values("InvoiceAmount")),
       y="Commodity", 
       x="InvoiceAmount",
       color="Category",
       orientation="h")
    .update_layout(height=600)
)

### Which months have the largest spend volume in which category?

It is often a good idea to balance cash flows by considering the timing of cash-out and cash-in events. Often, there are end-of-the-year and beginning-of-the-year payments that are poorly aligned with the cash-in situation. Especially repeated payables should be synchronized with receivables.

In [ ]:
px.bar(data_frame=(df[["Date","Category","Commodity","InvoiceAmount"]]
                     .assign(month=lambda x: x.Date.dt.month)
                     .assign(Month=lambda x: x.Date.dt.month_name())
                     .drop(["Date"], axis=1)
                     .groupby(["month","Category","Commodity"])
                     .agg({"Month":"first","InvoiceAmount":"sum"})
                     .reset_index()
                    ),
       x="Month", 
       y="InvoiceAmount", 
       color="Category", 
       hover_name="Commodity"
      )

## Item Spend Analysis

Item spend analysis refers to analyzing expenditures at item level. This analysis gives the ability to know whether a specific item is being purchased from multiple suppliers, from several locations, and at different item prices. Doing this analysis can highlight the different opportunities for purchasing on better terms and potentially identify spend leakage issues, such as purchasing from non-preferred vendors or buying from suppliers that do not fall under the organizations procurement policy (maverick spend). 

### What is the unit price of the 50 items with the highest purchase value?

Items with a high purchase value have a large leverage on the bottom line. It is therefore important to take a close look at prices and vendors underlying these items.

In [9]:
important_items = (df[["ItemID","InvoiceAmount"]]
                   .groupby("ItemID")
                   .sum()
                   .reset_index()
                   .sort_values("InvoiceAmount")
                   .iloc[-50:]
                  )
px.scatter(
    data_frame=(df
        .loc[lambda x : x.ItemID.isin(important_items.ItemID)]
        .loc[lambda x: x.InvoiceAmount>0]
        .assign(ItemID=lambda x:x.ItemID.astype(str))),
    x="ItemID", 
    y="UnitPrice",
    size="InvoiceAmount",
    color="Name"
)

## Tail Spend Analysis

Tail spend is defined as the those transcations that make up the lowest 20% of the total spend volume. With millions of transactions every year, these purchases that are small or infrequent can go overlooked, but tail spend is the place where procurement organizations may be leaving money on the table and utilizing their leverage inefficiently.

In [10]:
tail_spend = (df[["InvoiceID","Name","InvoiceAmount","Commodity"]]
     .loc[lambda x : x.InvoiceAmount>0]         
     .groupby(["InvoiceID","Commodity","Name"])
     .sum()
     .reset_index()
     .sort_values("InvoiceAmount")
     .loc[lambda x : x.InvoiceAmount.cumsum()<0.2*x.InvoiceAmount.sum()]
)
tail_spend

,InvoiceID,Commodity,Name,InvoiceAmount
87922,580203,Other,Techiit Incorporated,0.01
87950,580229,Other,Techiit Incorporated,0.01
87889,580172,Other,Techiit Incorporated,0.01
87885,580170,Other,Techiit Incorporated,0.01
87965,580242,Other,Techiit Incorporated,0.01
...,...,...,...,...
30915,505695,Fabrication,Techtax Corporation,6160.15
55968,537676,Print,Sanholdings LLC,6160.32
54877,536274,Fabrication,silicontech LLC,6161.20
46227,525170,Fabrication,Techtax Corporation,6161.84


### Which categories are contributing most to tail spend?

Let us take a look at the type of categories that are part of our tail spend. We can use this as proxy to understand which proportion of our tail spend is actually taken under contract.

In [ ]:
(px.bar(
    data_frame=(tail_spend
                .groupby(["Commodity"])
                     .sum()
                     .reset_index()
                     .sort_values("InvoiceAmount")),
       y="Commodity", 
       x="InvoiceAmount",
       orientation="h")
    .update_layout(height=700)
)

<ipython-input-39-6955d319a1be>:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



### What are the largest companies that are not taken under contract?

Companies that re not taken under contract do not have a dedicated category manager. We can identify those companies by filtering those that have *other* as category and then sort them by total purchase value.

In [ ]:
(px.bar(
    data_frame=(tail_spend.loc[lambda x: x.Commodity=="Other"]
                .groupby(["Name"])
                     .sum()
                     .reset_index()
                     .sort_values("InvoiceAmount")),
       y="Name", 
       x="InvoiceAmount",
       orientation="h")
    .update_layout(height=700)
)

<ipython-input-40-b44b12eaacf8>:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



## Payment Term Analysis

Payment term analysis explores opportunities for re-negotiating payment terms, such as invoice due dates and discounts, but it also helps to review payment patterns to identify common practices. Some suppliers reward early payment with discounts - but keep in mind that this also means lost interest on working capital (cash). The reverse also holds as suppliers who do not offer discounts should allow longer invoice due days.

### What are the most common invoice due days?

Before we begin let us take a look at what are the most common payment terms across the entire spend dataset.

In [11]:
px.bar(
    data_frame=(df[["InvoiceID","DueDays"]]
                .assign(DueDays=lambda x : x.DueDays.astype(str))
                .groupby(["DueDays"])
                .count()
                .reset_index()
                .sort_values("InvoiceID", ascending=False)),
    x="DueDays",
    y="InvoiceID"
)

### What is the potential for discounts?

We can scope the potential for discounts by looking for groups of contracts that have high invoice due days but zero discounts. The reverse is also true by looking for contracts that offer low discounts but have very short payment terms. Well negotiated terms show an inverse relationship between invoice due days and discounts. For example, invoices with less than 20 due days should offer discounts, whereas invoices with more than 60 days do not.

In [ ]:
px.scatter(
    data_frame=(df.loc[lambda x : x.DueDays<1000]
                .assign(count=lambda x: x.DueDays)
                .groupby(["Discount","DueDays"])
                .count()
                .reset_index()),
    x="DueDays", 
    y="Discount", 
    size="count"
)

## Exercises

1. If a site receives a lot of goods and services from another country that has a different currency, the site's business is exposed to exchange rate risk. Create bar chart that shows which sites are supplied from which country of origin. Which foreign currency poses the highest exchange rate risk for U.S. sites?

2. An important part of assessing the performance of the supplier base is to get an understanding on how fragmented the supplier base is within each category (here: commodity). Create a bar chart that shows the categorical distribution of the share spend by number of suppliers per category.